In [1]:
#!/ usr/bin/env python3 .7

# Copyright 2020 , Gurobi Optimization , LLC

# We find alternative epsilon - optimal solutions to a given knapsack
# problem by using PoolSearchMode

# 知识点

`PoolSearchMode`
> Selects different modes for exploring the MIP search tree. With the default setting `(PoolSearchMode=0)`,
the MIP solver tries to find an optimal solution to the model. It keeps other solutions found along
the way, but those are incidental.   

> By setting this parameter to a non-default value, the MIP search will continue after the optimal solution has been found in order to find additional, high-quality solutions.   

> With a setting of 2, it will find the n best solutions, where n is determined by the value
of the `PoolSolutions` parameter.   

> With a setting of 1, it will try to find additional solutions, but with no guarantees about the quality of those solutions. The cost of the solve will increase with increasing values of this parameter.  

`PoolSolutions`
> Number of MIP solutions to store

In [2]:
from __future__ import print_function
import gurobipy as gp
from gurobipy import GRB
import sys

In [3]:
# Sample data
Groundset = range (10)
objCoef = [32 , 32, 15, 15, 6, 6, 1, 1, 1, 1]
knapsackCoef = [16 , 16, 8, 8, 4, 4, 2, 2, 1, 1]
Budget = 33

In [4]:
# Create initial model
model = gp. Model (" poolsearch ")

Using license file /Users/yj/gurobi.lic


In [7]:
# Create dicts for tupledict . prod () function
objCoefDict = dict (zip ( Groundset , objCoef ))
knapsackCoefDict = dict (zip( Groundset , knapsackCoef ))
print(objCoefDict)
print(knapsackCoefDict)

{0: 32, 1: 32, 2: 15, 3: 15, 4: 6, 5: 6, 6: 1, 7: 1, 8: 1, 9: 1}
{0: 16, 1: 16, 2: 8, 3: 8, 4: 4, 5: 4, 6: 2, 7: 2, 8: 1, 9: 1}


In [9]:
# Initialize decision variables for ground set:
# x[e] == 1 if element e is chosen
Elem = model.addVars (Groundset, vtype=GRB.BINARY, name='El')
Elem

{0: <gurobi.Var *Awaiting Model Update*>,
 1: <gurobi.Var *Awaiting Model Update*>,
 2: <gurobi.Var *Awaiting Model Update*>,
 3: <gurobi.Var *Awaiting Model Update*>,
 4: <gurobi.Var *Awaiting Model Update*>,
 5: <gurobi.Var *Awaiting Model Update*>,
 6: <gurobi.Var *Awaiting Model Update*>,
 7: <gurobi.Var *Awaiting Model Update*>,
 8: <gurobi.Var *Awaiting Model Update*>,
 9: <gurobi.Var *Awaiting Model Update*>}

In [10]:
# Set objective function
model.ModelSense = GRB.MAXIMIZE
model.setObjective(Elem.prod(objCoefDict))

In [11]:
# Constraint : limit total number of elements to be picked to be at most
# Budget
model.addConstr(Elem.prod(knapsackCoefDict) <= Budget, name='Budget')

<gurobi.Constr *Awaiting Model Update*>

## Core part here

In [12]:
# Limit how many solutions to collect
model.setParam(GRB.Param.PoolSolutions, 1024)
# Limit the search space by setting a gap for the worst possible solution
# that will be accepted
model.setParam(GRB.Param.PoolGap, 0.1)
# do a systematic search for the k- best solutions
model.setParam(GRB.Param.PoolSearchMode, 2)

Changed value of parameter PoolSolutions to 1024
   Prev: 10  Min: 1  Max: 2000000000  Default: 10
Changed value of parameter PoolGap to 0.1
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Changed value of parameter PoolSearchMode to 2
   Prev: 0  Min: 0  Max: 2  Default: 0


In [14]:
# save problem
model.write ('poolsearch.rlp')

In [15]:
model.optimize()

Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (mac64)
Optimize a model with 1 rows, 20 columns and 10 nonzeros
Model fingerprint: 0x0f50c946
Variable types: 0 continuous, 20 integer (20 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+00, 3e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+01, 3e+01]
Found heuristic solution: objective 65.0000000
Presolve time: 0.05s
Presolved: 1 rows, 20 columns, 10 nonzeros
Variable types: 0 continuous, 20 integer (20 binary)

Root relaxation: objective 6.587500e+01, 1 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   65.87500    0    1   65.00000   65.87500  1.35%     -    0s

Optimal solution found at node 0 - now completing solution pool...

    Nodes    |    Current Node    |      Pool Obj. Bounds     |     Work
             |                    |   Worst

In [16]:
model.setParam(GRB.Param.OutputFlag, 0)

In [27]:
# Status checking
status = model.Status

if status in (GRB.INF_OR_UNBD, GRB.INFEASIBLE, GRB.UNBOUNDED):
    print('The model cannot be solved because it is infeasible or unbounded')
    sys.exit(1)
    
if status != GRB.OPTIMAL:
    print('Optimization was stopped with status ' + str(status))
    sys.exit(1)

# Print best selected set
print('Select elements in best solution:')
print('\t', end='')
for e in Groundset:
    if Elem[e].X > .9:
        print('E1%d' % e, end=' ')
print('')

# Print number of solution stored
nSolution = model.SolCount
print('number of solution stored:' + str(nSolution))

# Print objective values of solutions
for e in range (20):
    model.setParam(GRB.Param.SolutionNumber, e)
    print ('%g' % model.PoolObjVal, end=' ')

Select elements in best solution:
	E10 E11 E18 
number of solution stored:1024
65 65 65 65 65 65 65 65 65 65 65 65 65 65 65 65 65 65 65 65 